# Accessing TTree data with PyROOT (and others)

Very likely, most of your data are in ROOT TTrees. Here are various ways of getting the data out.

In [3]:
import ROOT

Welcome to JupyROOT 6.18/04


In [4]:
# Note: this is TFile::Open(...)
file = ROOT.TFile.Open("http://scikit-hep.org/uproot/examples/HZZ.root")
tree = file.Get("events")

In [5]:
for event in tree:
    print(event.MET_px, event.MET_py)

5.912771224975586 2.5636332035064697
24.76520347595215 -16.349109649658203
-25.78508758544922 16.237131118774414
8.619895935058594 -22.78654670715332
5.393138885498047 -1.3100523948669434
-3.7594752311706543 -19.417020797729492
23.962148666381836 -9.049156188964844
-57.533348083496094 -20.48767852783203
42.416194915771484 -94.35086059570312
-1.9144694805145264 -23.96303367614746
19.710058212280273 4.645508766174316
-35.538055419921875 -14.753822326660156
15.2099027633667 -6.3974995613098145
28.45915985107422 5.657520294189453
-9.093977928161621 0.8614864945411682
0.3503572940826416 -4.255575656890869
-0.20190052688121796 -3.49951434135437
8.286861419677734 -3.2661635875701904
-7.835591793060303 -13.773303031921387
-41.88085174560547 -157.98814392089844
-1.7157256603240967 19.48651885986328
2.3737385272979736 -8.089887619018555
-30.23499298095703 2.0271899700164795
13.711685180664062 -6.066513538360596
130.72396850585938 74.9982681274414
-6.254471778869629 18.144054412841797
-14.4688358

6.278502941131592 3.987032175064087
-7.625055313110352 -21.98479461669922
3.7786128520965576 6.70081090927124
10.144625663757324 -13.557449340820312
-4.58072566986084 -0.09167322516441345
4.600253582000732 2.5543816089630127
-37.56105422973633 4.451735496520996
6.159669876098633 -1.817726731300354
-1.89145028591156 -9.804818153381348
-43.02328872680664 26.546178817749023
-1.1438751220703125 13.790300369262695
-23.875347137451172 41.883934020996094
19.668058395385742 -33.086448669433594
-25.24504852294922 14.608850479125977
-7.251593112945557 -10.604716300964355
-80.2021713256836 36.72562026977539
4.55368709564209 6.0222907066345215
-16.640758514404297 7.002596855163574
21.500654220581055 -8.970060348510742
-12.613741874694824 -2.521346092224121
56.24713897705078 -17.508853912353516
-40.95276641845703 -59.68227005004883
15.719745635986328 -22.054752349853516
-1.023589015007019 -1.8625972270965576
-51.08091735839844 -55.19401931762695
1.464843511581421 -8.013097763061523
-2.7743456363677

That worked, but it would be uncomfortable on large datasets because expressions like `event.MET_px` does a lot of processing _in each event_ to produce a number. The same reasons to use Numpy over Python for loops applies here (to an even greater degree).

Fortunately, ROOT now has methods to access TTree data as Numpy arrays.

In [6]:
tree.AsMatrix(["MET_px", "MET_py"])

array([[  5.91277122,   2.5636332 ],
       [ 24.76520348, -16.34910965],
       [-25.78508759,  16.23713112],
       ...,
       [ 18.10164642,  50.29071808],
       [ 79.87519073, -52.35145187],
       [ 19.71374893,  -3.59541821]])

The labels (`MET_px`, `MET_py`) are missing, but here's a way to get them:

In [7]:
data, labels = tree.AsMatrix(["MET_px", "MET_py"], return_labels=True)
labels

['MET_px', 'MET_py']

And use them:

In [8]:
recarray = data.reshape(-1).view([(x, data.dtype) for x in labels])
recarray

array([(  5.91277122,   2.5636332 ), ( 24.76520348, -16.34910965),
       (-25.78508759,  16.23713112), ..., ( 18.10164642,  50.29071808),
       ( 79.87519073, -52.35145187), ( 19.71374893,  -3.59541821)],
      dtype=[('MET_px', '<f8'), ('MET_py', '<f8')])

In [9]:
recarray["MET_px"]

array([  5.91277122,  24.76520348, -25.78508759, ...,  18.10164642,
        79.87519073,  19.71374893])

In [10]:
recarray["MET_py"]

array([  2.5636332 , -16.34910965,  16.23713112, ...,  50.29071808,
       -52.35145187,  -3.59541821])

## Nested structure

Not all data have one value per event/entry.

In [12]:
for event in tree:
    print([x for x in event.Muon_Px])

[-52.89945602416992, 37.7377815246582]
[-0.8164593577384949]
[48.987831115722656, 0.8275666832923889]
[22.08833122253418, 76.6919174194336]
[45.171321868896484, 39.75095748901367]
[9.228110313415527, -5.793715000152588]
[12.538717269897461, 29.541839599609375]
[34.883758544921875]
[-53.16697311401367, 11.491869926452637]
[-67.01485443115234, -18.118755340576172]
[15.983028411865234, 34.68440628051758]
[-70.51190948486328, -38.028743743896484]
[58.94381332397461]
[-15.587870597839355]
[-122.33011627197266, -1.0597527027130127]
[-46.70415496826172, 39.020023345947266]
[51.29465866088867, 17.45092010498047]
[43.28120040893555]
[-45.92393493652344, 22.549766540527344]
[43.29360580444336, -33.28158187866211, -4.376191139221191]
[-27.43815040588379, 28.944164276123047]
[-24.200502395629883, 26.48993492126465]
[41.23143005371094, -39.87832260131836]
[-29.219207763671875]
[-61.25238037109375]
[7.968410968780518, -15.08200454711914]
[8.993956565856934]
[-28.640575408935547]
[22.636903762817383,

[55.79229736328125, 43.48860168457031]
[1.178104043006897, -42.05720138549805]
[-145.9373016357422]
[-45.84864807128906]
[43.51326370239258, -33.02598190307617]
[-26.22919273376465, 48.16505432128906]
[64.47379302978516, -3.7717783451080322]
[-26.042463302612305]
[-13.344138145446777]
[31.04361915588379]
[17.846176147460938, -32.586727142333984]
[17.753297805786133, -42.451019287109375]
[-24.20648956298828, 5.404818058013916]
[38.155418395996094, -4.860474109649658]
[-7.493404865264893, -31.862009048461914]
[-41.8912353515625]
[9.928046226501465]
[-13.550823211669922]
[-10.456878662109375]
[-78.62123107910156, 27.916162490844727]
[-44.135704040527344, 1.1296159029006958]
[1.1599587202072144]
[31.915191650390625]
[-32.80454635620117, 39.13153839111328]
[-11.94250202178955, 20.28976821899414]
[-33.24270248413086]
[20.95574378967285, 1.900225043296814]
[-12.44887638092041]
[-13.188936233520508, -21.775360107421875]
[6.10626220703125, 55.33865737915039]
[152.44821166992188, 32.498367309570

These kinds of data don't have a Numpy analogue, so `AsMatrix` can't read them.

In [13]:
tree.AsMatrix(["Muon_Px"])

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

Error in <TTreeReaderValueBase::GetBranchDataType()>: Must use TTreeReaderArray to read branch Muon_Px: it contains an array or a collection.
Error in <TTreeReaderValueBase::CreateProxy()>: The branch Muon_Px contains data of type {UNDETERMINED TYPE}, which does not have a dictionary.


ROOT doesn't (yet) have a method for this, but two Python packages, root_numpy and uproot, do (check lecture about uproot this afternoon with Mason).

## Arbitrary objects

PyROOT should be able to read any kind of object, as it's a front end for ROOT.

In [14]:
file2 = ROOT.TFile.Open("http://scikit-hep.org/uproot/examples/HZZ-objects.root")
tree3 = file2.Get("events")

In [ ]:
for event in tree3:
    print(event.muonp4, [x.Pt() for x in event.muonp4])

{ @0x55d1fc597930, @0x55d1fc597970 } [54.16810703140204, 37.74415265978988]
{ @0x55d1fc597930 } [24.41791248135961]
{ @0x55d1fc597930, @0x55d1fc597970 } [53.58826697278532, 29.811997139120674]
{ @0x55d1fc597930, @0x55d1fc597970 } [88.63194310332618, 77.95148447265454]
{ @0x55d1fc597930, @0x55d1fc597970 } [81.01140452083426, 47.17504574667714]
{ @0x55d1fc597930, @0x55d1fc597970 } [41.591052358050575, 30.844215085173435]
{ @0x55d1fc597930, @0x55d1fc597970 } [44.35777431661225, 29.874452374029385]
{ @0x55d1fc597930 } [38.37087544930784]
{ @0x55d1fc597930, @0x55d1fc597970 } [106.28355631699014, 12.311635885539262]
{ @0x55d1fc597930, @0x55d1fc597970 } [85.53881159059787, 39.506103898783294]
{ @0x55d1fc597930, @0x55d1fc597970 } [51.64914852288953, 46.49716868294783]
{ @0x55d1fc597930, @0x55d1fc597970 } [75.51093887887036, 50.514888290524446]
{ @0x55d1fc597930 } [79.79030751726177]
{ @0x55d1fc597930 } [26.225671131477768]
{ @0x55d1fc597930, @0x55d1fc597970 } [128.02862618591368, 49.6202390526

{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [27.813852527789823, 15.338240759128794]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [95.50074346350422, 27.45730307247635]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [93.96306992179517, 58.740293637115784]
{ @0x55d1fdd04d80 } [97.37615837816877]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [68.02864977404299, 29.12398709232859]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [52.23626023456625, 48.00561017711447]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [42.48831988699897, 27.224808954334584]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [96.43111085784263, 28.74598086139535]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0, @0x55d1fdd04e00 } [65.43590886109413, 44.9402807060488, 20.367365089514266]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [53.535266033086224, 39.05810978114802]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [39.40205258917391, 34.6231733859597]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [96.04242726257486, 45.7151953198263]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [45.15620028377504, 43.4702295788937]
{ @0x55d1f

{ @0x55d1fdd04d80 } [48.878533164756]
{ @0x55d1fdd04d80 } [38.1863645832055]
{ @0x55d1fdd04d80 } [79.94541329640661]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [67.48973169266586, 28.801927210633085]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0, @0x55d1fdd04e00 } [53.3293986017559, 30.829012464113948, 25.13169525325904]
{ @0x55d1fdd04d80 } [29.53701937573131]
{ @0x55d1fdd04d80 } [30.27608345698622]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [51.132744909104225, 33.20828131937303]
{ @0x55d1fdd04d80 } [77.7359352727952]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [27.71291666447939, 22.350128476301943]
{ @0x55d1fdd04d80 } [37.079919035837676]
{ @0x55d1fdd04d80 } [41.68264751610324]
{ @0x55d1fdd04d80 } [130.12969708652773]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [75.2899538519269, 42.33814185135061]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [53.067887441020964, 42.607306510177075]
{ @0x55d1fdd04d80 } [60.17158708846729]
{ @0x55d1fdd04d80 } [44.39186051568234]
{ @0x55d1fdd04d80 } [24.562162819885295]
{ @0x55d1fdd04d80, @0x55d

{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [55.37892901623524, 54.446940739151124]
{} []
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [69.75838611744507, 27.77829740822057]
{ @0x55d1fdd04d80 } [56.390351070249636]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [87.03467375522708, 40.27796589733983]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [72.4070279865537, 15.35757519282291]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [32.34628651421709, 30.02023885242416]
{ @0x55d1fdd04d80 } [31.99106093944338]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [41.969725279398716, 33.87146589172187]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [43.02525837118548, 36.906735971878476]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [51.213258739128655, 39.997867234536265]
{ @0x55d1fdd04d80 } [309.5895917507714]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [43.41947663905803, 40.80852878873518]
{ @0x55d1fdd04d80 } [88.86372148427292]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [47.13414539856727, 45.89183397485163]
{ @0x55d1fdd04d80 } [59.81587612158034]
{ @0x55d1fdd04d80 } [24.71715

{ @0x55d1fdd04d80 } [217.2985028397765]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [53.73359604011407, 10.581938157343941]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [53.079062988689905, 36.657772892260546]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [80.41298735657996, 20.22346672563356]
{ @0x55d1fdd04d80 } [79.09790280221969]
{ @0x55d1fdd04d80 } [49.85805824200854]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [110.2466286714445, 39.27817916705539]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [60.01382842986951, 33.03202077057517]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [66.56028827512726, 28.246934819775024]
{ @0x55d1fdd04d80 } [67.43918972229451]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [48.555452265893315, 32.8074881148849]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [59.82019954595482, 50.66328043015607]
{ @0x55d1fdd04d80 } [53.40235981480932]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [27.29825615983026, 18.926222473227977]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [74.76210982278477, 44.11048813022025]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 

{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [79.47245619104167, 38.09255159281517]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [43.73230262463395, 37.239113584045434]
{ @0x55d1fdd04d80 } [54.9747697850464]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [62.506474669137084, 13.56100651964766]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0, @0x55d1fdd04e00, @0x55d1fdd04e40 } [43.86202831432053, 41.82392376605607, 35.04765323756469, 10.80854569485194]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [37.891795723284176, 24.597622085864955]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [51.80018127169638, 32.84951457604351]
{ @0x55d1fdd04d80 } [26.84249328342595]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [114.66813185778047, 21.59444252507114]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [87.29691340464076, 78.30823937620374]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [55.8710823140808, 25.166654888061395]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [67.16522354437552, 37.28865542561307]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [58.17880738346228, 28.626719257850613]
{ @0x55

{ @0x55d1fdd04d80 } [30.488762955447367]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [54.51182404418045, 25.897553524027035]
{ @0x55d1fdd04d80 } [84.65700135034473]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [44.425013337226645, 30.389003775298676]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [61.540656978693896, 22.549259817970384]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [95.45549168425929, 19.94099003760087]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [65.07505715990824, 14.027040462735249]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [61.168416288927695, 26.683483511075657]
{} []
{ @0x55d1fdd04d80 } [68.4364760829509]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [49.01329795437956, 47.647243997466255]
{ @0x55d1fdd04d80 } [89.51878044487759]
{ @0x55d1fdd04d80 } [79.3335087297301]
{ @0x55d1fdd04d80 } [56.58736585156769]
{ @0x55d1fdd04d80 } [42.8419885016009]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [63.518651223137596, 29.305796525171893]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [80.79000911610511, 31.093063616387003]
{ @0x55d1fdd04d80, @

{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [73.23966918595787, 29.504913672934187]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [100.97271441229114, 46.13557917595929]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [61.62028459402537, 27.32927259746756]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [67.87946718622091, 41.97066537188601]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [55.182008015035, 38.56795540031617]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [52.07113211389906, 40.769692769449094]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [72.99414320262385, 22.84392436474503]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [67.9947836414484, 28.132265074798973]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [152.3158844076997, 36.34722440347388]
{ @0x55d1fdd04d80 } [43.9220109129492]
{ @0x55d1fdd04d80 } [26.80760927901637]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [68.19661773463075, 28.774402130651257]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [47.26976595541939, 39.25142403070173]
{ @0x55d1fdd04d80 } [86.51292362112734]
{ @0x55d1fdd04d80, @0x55d1fdd04dc0 } [78.40

## Performance


In [ ]:
%%timeit
tree3.array("Muon_Px")

## Summary

Which method you choose depends on accessibility of ROOT, what types of data you need to read, and the scale of your dataset.

<table style="font-size: 22pt; margin-top: 50px">
    <tr style="font-weight: bold"><td>Method</td><td>Relationship to ROOT</td><td>Data types</td><td>Performance</td></tr>
    <tr><td>PyROOT for loop</td><td>part of ROOT</td><td>any</td><td>slow</td></tr>
    <tr><td>PyROOT AsMatrix</td><td>part of ROOT</td><td>one number per entry</td><td>fast</td></tr>
    <tr><td>uproot</td><td>independent implementation</td><td>number(s) per entry, most objects</td><td>fast</td></tr>
</table>